# TP3
### Problema 1 


Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $(n,m)\,$ na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $n$ e $m$ à sua escolha.
    
Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos usando Inteiros  (apresentado no TP2).


### Implementação
Comecemos por importar a biblioteca `pysmt.shortcuts` que disponibiliza funcionalidades para a utilização de um SMT solver, assim como, a biblioteca `pysmt.typing` de onde temos que importar o tipo `INT`.

In [4]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools 

Para modelar este programa como um SFOTS teremos o conjunto $\mathsf{X}$ de variáveis do estado dado pela lista `['pc', 'x', 'y', 'z']`, e definimos a função
`genState` que recebe a lista com o nome das variáveis do estado, uma etiqueta e um inteiro, e cria a i-ésima cópia das variáveis do estado para essa etiqueta.

In [5]:
# declaração das variáveis 
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

A função `inicializacao` testa se um dado estado cumpre a condição para ser estado inicial.

In [6]:
def inicializacao(state, a, b):
    return And(Equals(state['nodo'], Int(0)), 
               Equals(state['z'], Int(0)),
               Equals(state['x'], Int(a)),
               Equals(state['y'], Int(b)))

De seguida, definimos as funções `odd` e `even` que verificam se um dado número passado como argumento é ímpar ou par, respetivamente. Definimos também as funções `overflow1` e `overflow2` que verificam se um dado estado do programa não respeita a restrição relativa ao número máximo de bits que uma variável pode tomar.

In [7]:
# alterar - nao usar bitvec
def odd(curr):
    return NotEquals(Times(Div(curr['y'], Int(2)), Int(2)), curr['y'])
    

def even(curr):
    return Equals(Times(Div(curr['y'], Int(2)), Int(2)), curr['y'])
    

# estados de erro 
# n é o inteiro máximo que pode atingir 
def overflow1(curr, n):
    return GT(Times(curr['x'], Int(2)), Int(n))


def overflow2(curr, n):
    return GT(Plus(curr['z'], curr['x']), Int(n))

A função `error` testa se um dado estado é um possível estado de erro do programa.

In [8]:
def error(state):
    return Or(Equals(state['nodo'], Int(3)), Equals(state['nodo'], Int(5)))

Dados dois estados, a função `transicao` testa se é possível transitar do primeiro estado para o segundo estado.

In [9]:
def transicao(curr, prox, n):
    c1 = And(
            Equals(curr['nodo'], Int(0)),
            NotEquals(curr['y'], Int(0)),
            even(curr),
            Equals(prox['nodo'], Int(1)),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c2 = And(
            Equals(curr['nodo'], Int(0)),
            NotEquals(curr['y'], Int(0)),
            odd(curr),
            Equals(prox['nodo'], Int(2)),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c3 = And(
            Equals(curr['nodo'], Int(0)),
            Equals(curr['y'], Int(0)),
            Equals(prox['nodo'], Int(4)),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c4 = And(
            Equals(curr['nodo'], Int(1)),
            Equals(prox['nodo'], Int(0)),
            Not(overflow1(curr, n)),
            Equals(prox['x'], Times(curr['x'], Int(2))),
            Equals(prox['y'], Div(curr['y'], Int(2))),
            Equals(prox['z'], curr['z']))
    
    c5 = And(
            Equals(curr['nodo'], Int(1)),
            Equals(prox['nodo'], Int(3)),
            overflow1(curr, n),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c6 = And(
            Equals(curr['nodo'], Int(2)),
            Equals(prox['nodo'], Int(0)),
            Not(overflow2(curr, n)),
            Equals(prox['z'], Plus(curr['z'], curr['x'])),
            Equals(prox['y'], Minus(curr['y'], Int(1))),
            Equals(prox['x'], curr['x']))
    
    c7 = And(
            Equals(curr['nodo'], Int(2)),
            Equals(prox['nodo'], Int(5)),
            overflow2(curr, n),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c8 = And(
            Equals(curr['nodo'], Int(3)),
            Equals(prox['nodo'], Int(3)),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c9 = And(
            Equals(curr['nodo'], Int(4)),
            Equals(prox['nodo'], Int(4)),
            Equals(prox['x'], curr['x']),
            Equals(prox['y'], curr['y']),
            Equals(prox['z'], curr['z']))
    
    c10 = And(Equals(curr['nodo'], Int(5)),
             Equals(prox['nodo'], Int(5)),
             Equals(prox['x'], curr['x']),
             Equals(prox['y'], curr['y']),
             Equals(prox['z'], curr['z']))
   
    return Or(c1, c2, c3, c4, c5, c6, c7, c8, c9, c10)

A função `genTrace` gera um possível traço de execução do programa com $n$ transições.

In [10]:
def genTrace(vars, init, trans, error1, error2, a, b, max_val, n):
    # n -> tamanho do traço de execução 
    # max_val -> valor máximo que pode atingir 
    with Solver(name="z3") as s:
        
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0], a, b)
        Tks = [ trans(X[i],X[i+1], max_val) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))
                    
genTrace(['nodo', 'x', 'y', 'z'], inicializacao, transicao, overflow1, overflow2, 5, 3, 30, 10)

Estado: 0
           nodo = 0
           x = 5
           y = 3
           z = 0
Estado: 1
           nodo = 2
           x = 5
           y = 3
           z = 0
Estado: 2
           nodo = 0
           x = 5
           y = 2
           z = 5
Estado: 3
           nodo = 1
           x = 5
           y = 2
           z = 5
Estado: 4
           nodo = 0
           x = 10
           y = 1
           z = 5
Estado: 5
           nodo = 2
           x = 10
           y = 1
           z = 5
Estado: 6
           nodo = 0
           x = 10
           y = 0
           z = 15
Estado: 7
           nodo = 4
           x = 10
           y = 0
           z = 15
Estado: 8
           nodo = 4
           x = 10
           y = 0
           z = 15
Estado: 9
           nodo = 4
           x = 10
           y = 0
           z = 15


A função `invert` recebe como argumento a função que codifica a relação de transição, devolvendo a sua inversa.

In [11]:
# trans(u,v)^-1 = trans(v,u) 
def invert(trans, curr, prox, n):
    return trans(prox, curr, n)

### Model Cheking

O algoritmo de “model-checking” manipula as fórmulas $\;\mathsf{R}_n\;\equiv\; \mathsf{I}\,\land\,\mathsf{T}^n\;$ e $\;\mathsf{U}_m\equiv\; \mathsf{E}\,\land\,\mathsf{B}^m\;$ fazendo crescer os índices $\;n,m\;$ de acordo com determinadas regras.

Para auxiliar na implementação deste algoritmo, começamos por definir duas funções.
A função `rename` renomeia uma fórmula (sobre um estado) de acordo com um dado estado. 
A função `same` testa se dois estados são iguais.

In [12]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

In [19]:
def model_checking(vars, init, trans, error, a, b, max_val, N, M):
    with Solver(name="z3") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        for (n,m) in order:
            I = init(X[0], a, b)
            Tn = And([trans(X[i], X[i+1], max_val) for i in range(n)])
            Rn = And(I, Tn)
            
            Bm = And([invert(trans, Y[i], Y[i+1], max_val) for i in range(m)])
            E = error(Y[0])
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um) 
            
            if s.solve([Vnm]):
                print("unsafe")
                return 
            
            else: # Vnm insatisfazível
                C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
                
                if C is None:
                    print("interpolant None")
                    continue
                    
                C0 = rename(C, X[0])
                C1 = rename(C, X[1])
                T = And(X[0], X[1])
                
                if not s.solve([C0, T, Not(C1)]): # C é invariante de T
                    print("safe")
                    return
                
                else: # tenta gerar o majorante S
                    S = rename(C, X[n])
                    while True:
                        A = And(S, trans, X[n], Y[m])
                        if s. solve(A, Um):
                            print("Não é possivel encontrar um majorante")
                            break
                        
                        else:
                            Cnew = binary_interpolant(A, Um)
                            Cn = rename(Cnew, X[n])
                            
                            if s.solve([Cn, Not(S)]): # se Cn->S não é tautologia
                                S = Or(S, Cn)
                                
                            else: # S foi encontado
                                print("safe")
                                return 
            
              
        print("unknown")                   
        

model_checking(['nodo', 'x', 'y', 'z'], inicializacao, transicao, error, 5, 3, 30, 50, 50)      

ConvertExpressionError: Could not convert the input expression. The formula contains unsupported operators. The error was: Unsupported operator 'DIV' (node_type: 62)

In [26]:
def model_checking(vars, init, trans, error, a, b, max_val, N, M):
    
    with Solver(name="z3") as s:
        
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1])
        
        for (n,m) in order:
            I = inicializacao(X[0], a, b)
            Tn = And([transicao(X[i], X[i+1], max_val) for i in range(n)])
            Rn = And(I, Tn)
            
            Bm = And([invert(trans, Y[i], Y[i+1], max_val) for i in range(m)])
            E = error(Y[0])
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
            
            if s.solve([Vnm]):
                print("unsafe")
                return
            else:
                while n < N or m < M:
                    C = binary_interpolant(And(Rn, same(X[n], Y[m])), Um)
                    if C is None:
                        print("interpolant None")
                        choice = str(input("Que variável incrementar? "))
                        if choice == 'm':
                            m += 1
                        elif choice == 'n':
                            n += 1
                    else:
                        C0 = rename(C, X[0])
                        C1 = rename(C, X[1])
                        T = trans1(X[0], X[1])
                        if not s.solve([C0,T,Not(C1)]):
                            print("safe")
                            return
                    
            

In [25]:
model_checking(['nodo', 'x', 'y', 'z'], inicializacao, transicao, error, 5, 3, 30, 50, 50)

ConvertExpressionError: Could not convert the input expression. The formula contains unsupported operators. The error was: Unsupported operator 'DIV' (node_type: 62)